## NCC 학습 데이터의 어노테이션을 추출
__데이터셋 번호가 180116 데이터와 반대!__  
__(1월 데이터에서는 채인혜선생님이 1, 김윤주선생님이 2를 하셨다면 5월엔 채인혜선생님이 2, 김윤주선생님이 1이라는 뜻)__  
1번 데이터셋은 붉은색 (120, 33, 33 정도) -> 기존 어노테이션 활용  
2번 데이터셋은 초록색 (0, 255, 0) -> 새로 어노테이션 함  

In [1]:
import pandas as pd
import cv2
import os
from glob import glob
import pydicom as dicom
import numpy as np
from matplotlib import pyplot as plt
from IPython.core.debugger import set_trace

%matplotlib inline

In [2]:
ncc_base_path = "/home/huray/data/NCC_trainset"
dicom_base_path = os.path.join(ncc_base_path, "dicom")

img_save_path = dicom_base_path.replace('dicom', 'img_retinanet')

WIDTH = 1400
HEIGHT = 1750

In [3]:
import shutil
import os

files = sorted(glob("/home/huray/data/NCC_trainset/dicom/abn_180517_*/*.dcm", recursive=True))
print(files[:5])
print(len(files))

['/home/huray/data/NCC_trainset/dicom/abn_180517_1/20000686_0001.dcm', '/home/huray/data/NCC_trainset/dicom/abn_180517_1/20000686_0002.dcm', '/home/huray/data/NCC_trainset/dicom/abn_180517_1/20000686_0003.dcm', '/home/huray/data/NCC_trainset/dicom/abn_180517_1/20000686_0004.dcm', '/home/huray/data/NCC_trainset/dicom/abn_180517_1/20255044_0001.dcm']
832


In [4]:
def detect_anno_set2(file_path):
    file_path = file_path.replace('.dcm', '.jpg')
    mask_arr = cv2.imread(file_path)
#     print(file_path)
    lower=np.array([0,252,0],np.uint8)
    upper=np.array([3,255,3],np.uint8)
    
    mask = cv2.inRange(mask_arr,lower,upper)
    
    (img_y, img_x, _) = mask_arr.shape
    
    scale_y = HEIGHT/img_y
    scale_x = WIDTH/img_x
    
    ret, thresh = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)
    _image, contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    annotation_counter = 0
    
    mask_arr = cv2.resize(mask_arr, (0, 0), fx=scale_x, fy=scale_y)
    
    anno_list = []
    
    for index, c in enumerate(contours):
#         print('contour index: ', index)
#         if index >= 1:
#             print('more than one contour!!!!')
        x, y, w, h = cv2.boundingRect(c)
        x = int(x*scale_x)
        w = int(w*scale_x)
        y = int(y*scale_y)
        h = int(h*scale_y)
        anno_list.append((x,y,x+w,y+h))
#         cv2.rectangle(mask_arr, (x,y), (x+w,y+h), (255, 0, 255), 10)
        
#     plt.figure(figsize=(20, 20))
#     plt.imshow(mask_arr)
#     plt.title('image')
#     plt.show()

    return anno_list

In [5]:
def detect_anno_set1(file_path):
    file_path = file_path.replace('.dcm', '.jpg')
    mask_arr = cv2.imread(file_path)
#     print(file_path)
    
    # 1차 검출 -> 적당한 범위로 어노테이션을 찾아서(컨투어 여러개 검출됨) 사각형을 하나 그림
    lower=np.array([20,20,120],np.uint8)
    upper=np.array([60,60,200],np.uint8)
    mask = cv2.inRange(mask_arr,lower,upper)
    
    ret, thresh = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)
    _image, contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    annotation_counter = 0
    
    for index, c in enumerate(contours):
        x, y, w, h = cv2.boundingRect(c)
        cv2.rectangle(mask_arr, (x,y), (x+w,y+h), (0, 255, 0), 8)
        
    # 2차 검출 -> 위에서 그린 사각형으로 정확한 어노테이션만 검출.
    lower=np.array([0,252,0],np.uint8)
    upper=np.array([3,255,3],np.uint8)
    mask = cv2.inRange(mask_arr,lower,upper)
        
    (img_y, img_x, _) = mask_arr.shape
    
    scale_y = HEIGHT/img_y
    scale_x = WIDTH/img_x    
        
    mask_arr = cv2.resize(mask_arr, (0, 0), fx=scale_x, fy=scale_y)
    
    ret, thresh = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)
    _image, contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    annotation_counter = 0
    
    anno_list = []
    
    for index, c in enumerate(contours):
#         print('contour index: ', index)
#         if index >= 1:
#             print('more than one contour!!!!')
        x, y, w, h = cv2.boundingRect(c)
        x = int(x*scale_x)
        w = int(w*scale_x)
        y = int(y*scale_y)
        h = int(h*scale_y)
        anno_list.append((x,y,x+w,y+h))
#         cv2.rectangle(mask_arr, (x,y), (x+w,y+h), (255, 0, 255), 7)
#         print(x,y,w,h)
    
#     plt.figure(figsize=(8, 8))
#     plt.imshow(mask_arr)
#     plt.title('image')
#     plt.show()
    
    return anno_list

In [6]:
img_path = []
x1 = []
y1 = []
x2 = []
y2 = []
class_name = []

errored_data_counter = 0
files_with_wrong_size = []

# continuer = True

for mammo_path in files:
#     if '180116_2/33250647_0001' in mammo_path:
#         continuer = False
    
#     if continuer:
#         continue

    print(mammo_path)
    mammo_dcm = dicom.read_file(mammo_path)
    
    if os.path.getsize(mammo_path)/(1024*1024) <= 1: # dcm파일이 1메가보다 작으면 pass
        files_with_wrong_size.append(mammo_path)
        print('DCM file is too small ' + mammo_path)
        continue
    
    try:
        mammo_arr = mammo_dcm.pixel_array
        mammo_arr = mammo_arr.astype(np.uint16)
    except (AttributeError, NotImplementedError) as e: # 종종 파일 자체가 문제가 있는 경우 있음.
        try:
            pixel_data = mammo_dcm[0x7fe0,0x0010].value # 파일 자체에 저장된 pixel_data값
            rows = mammo_dcm[0x0028, 0x0010].value # metadata로 들어있는 row
            cols = mammo_dcm[0x0028, 0x0011].value # metadata로 들어있는col

            mammo_arr = np.fromstring(pixel_data[:-1], dtype=np.uint16)
            mammo_arr = np.reshape(mammo_arr, (rows, cols))
        except ValueError:
            print('corrupted file: ' + mammo_path[:70])
            errored_data_counter += 1
            continue
        else:
            print('Attribute error" ' + mammo_path[:70])
    except Exception as e:
        print('different error: ' + mammo_path[:70])
        print(e)
        errored_data_counter += 1
        raise
        continue
        
    if len(mammo_arr.shape) == 3:
        print('RGB dicom!?')
        mammo_arr = cv2.cvtColor(mammo_arr, cv2.COLOR_BGR2GRAY)
        
    if mammo_arr.shape[0] <= 1800 and mammo_arr.shape[1] <= 1800:
        continue
        
    mammo_arr_final = (mammo_arr - np.amin(mammo_arr))/(np.amax(mammo_arr) - np.amin(mammo_arr)) * 255
    mammo_arr_final = mammo_arr_final.astype(np.uint8)
    mammo_arr_final = cv2.resize(mammo_arr_final, (WIDTH, HEIGHT))
    mammo_arr_final = np.asarray(np.dstack((mammo_arr_final, mammo_arr_final, mammo_arr_final)), dtype=np.uint8)

    anno_list = []
    
    if 'abn_180517_1' in mammo_path:
        anno_list = detect_anno_set1(mammo_path)
        data_sub_path = 'abn_180517_1'
    elif 'abn_180517_2' in mammo_path:
        anno_list = detect_anno_set2(mammo_path)
        data_sub_path = 'abn_180517_2'
    else:
        print('WRONG PATH!')
        raise
        
#     break
#     set_trace()
    
    _split = mammo_path.split('/')
    img_file_name = '{}'.format(_split[-1].replace('.dcm', ''))
    
    if len(anno_list) == 0:
        mammo_jpg_path = os.path.join(img_save_path, data_sub_path, img_file_name+'_0.jpg')
        cv2.imwrite(mammo_jpg_path, mammo_arr_final)
        
        img_path.append(mammo_jpg_path)
        x1.append('')
        y1.append('')
        x2.append('')
        y2.append('')
        class_name.append('')
    else:
        for i, anno_coord in enumerate(anno_list):
            mammo_jpg_path = os.path.join(img_save_path, data_sub_path, img_file_name+'_'+str(i)+'.jpg')
            cv2.imwrite(mammo_jpg_path, mammo_arr_final)

            img_path.append(mammo_jpg_path)
            x1.append(anno_coord[0])
            y1.append(anno_coord[1])
            x2.append(anno_coord[2])
            y2.append(anno_coord[3])
            class_name.append('M')
        
#     print(img_path, x1, y1, x2, y2)

/home/huray/data/NCC_trainset/dicom/abn_180517_1/20000686_0001.dcm
/home/huray/data/NCC_trainset/dicom/abn_180517_1/20000686_0002.dcm
/home/huray/data/NCC_trainset/dicom/abn_180517_1/20000686_0003.dcm
/home/huray/data/NCC_trainset/dicom/abn_180517_1/20000686_0004.dcm
/home/huray/data/NCC_trainset/dicom/abn_180517_1/20255044_0001.dcm
/home/huray/data/NCC_trainset/dicom/abn_180517_1/20255044_0002.dcm
/home/huray/data/NCC_trainset/dicom/abn_180517_1/20255044_0003.dcm
/home/huray/data/NCC_trainset/dicom/abn_180517_1/20255044_0004.dcm
/home/huray/data/NCC_trainset/dicom/abn_180517_1/30114690_0001.dcm
/home/huray/data/NCC_trainset/dicom/abn_180517_1/30114690_0002.dcm
/home/huray/data/NCC_trainset/dicom/abn_180517_1/30114690_0003.dcm
/home/huray/data/NCC_trainset/dicom/abn_180517_1/30114690_0004.dcm
/home/huray/data/NCC_trainset/dicom/abn_180517_1/30892022_0001.dcm
/home/huray/data/NCC_trainset/dicom/abn_180517_1/30892022_0002.dcm
/home/huray/data/NCC_trainset/dicom/abn_180517_1/30892022_0003

/home/huray/data/NCC_trainset/dicom/abn_180517_1/33250353_0003.dcm
/home/huray/data/NCC_trainset/dicom/abn_180517_1/33250353_0004.dcm
/home/huray/data/NCC_trainset/dicom/abn_180517_1/33250353_0004_1.dcm
/home/huray/data/NCC_trainset/dicom/abn_180517_1/33250506_0001.dcm
/home/huray/data/NCC_trainset/dicom/abn_180517_1/33250506_0002.dcm
/home/huray/data/NCC_trainset/dicom/abn_180517_1/33250506_0003.dcm
/home/huray/data/NCC_trainset/dicom/abn_180517_1/33250506_0004.dcm
/home/huray/data/NCC_trainset/dicom/abn_180517_1/33250822_0001.dcm
/home/huray/data/NCC_trainset/dicom/abn_180517_1/33250822_0002.dcm
/home/huray/data/NCC_trainset/dicom/abn_180517_1/33250822_0003.dcm
/home/huray/data/NCC_trainset/dicom/abn_180517_1/33250822_0004.dcm
/home/huray/data/NCC_trainset/dicom/abn_180517_1/33251049_0001.dcm
/home/huray/data/NCC_trainset/dicom/abn_180517_1/33251049_0002.dcm
/home/huray/data/NCC_trainset/dicom/abn_180517_1/33251049_0003.dcm
/home/huray/data/NCC_trainset/dicom/abn_180517_1/33251049_00

/home/huray/data/NCC_trainset/dicom/abn_180517_1/33255232_0001.dcm
/home/huray/data/NCC_trainset/dicom/abn_180517_1/33255232_0002.dcm
/home/huray/data/NCC_trainset/dicom/abn_180517_1/33255232_0003.dcm
/home/huray/data/NCC_trainset/dicom/abn_180517_1/33255232_0004.dcm
/home/huray/data/NCC_trainset/dicom/abn_180517_1/33255322_0001.dcm
/home/huray/data/NCC_trainset/dicom/abn_180517_1/33255322_0002.dcm
/home/huray/data/NCC_trainset/dicom/abn_180517_1/33255322_0003.dcm
/home/huray/data/NCC_trainset/dicom/abn_180517_1/33255322_0004.dcm
/home/huray/data/NCC_trainset/dicom/abn_180517_1/33255633_0001.dcm
/home/huray/data/NCC_trainset/dicom/abn_180517_1/33255633_0002.dcm
/home/huray/data/NCC_trainset/dicom/abn_180517_1/33255633_0003.dcm
/home/huray/data/NCC_trainset/dicom/abn_180517_1/33255633_0004.dcm
/home/huray/data/NCC_trainset/dicom/abn_180517_1/33256390_0001.dcm
/home/huray/data/NCC_trainset/dicom/abn_180517_1/33256390_0002.dcm
/home/huray/data/NCC_trainset/dicom/abn_180517_1/33256390_0003

/home/huray/data/NCC_trainset/dicom/abn_180517_1/33294826_0001.dcm
/home/huray/data/NCC_trainset/dicom/abn_180517_1/33294826_0002.dcm
/home/huray/data/NCC_trainset/dicom/abn_180517_1/33294826_0003.dcm
/home/huray/data/NCC_trainset/dicom/abn_180517_1/33294826_0004.dcm
/home/huray/data/NCC_trainset/dicom/abn_180517_1/33294886_0001.dcm
/home/huray/data/NCC_trainset/dicom/abn_180517_1/33294886_0002.dcm
/home/huray/data/NCC_trainset/dicom/abn_180517_1/33294886_0003.dcm
/home/huray/data/NCC_trainset/dicom/abn_180517_1/33294886_0004.dcm
/home/huray/data/NCC_trainset/dicom/abn_180517_1/33295029_0001.dcm
/home/huray/data/NCC_trainset/dicom/abn_180517_1/33295029_0002.dcm
/home/huray/data/NCC_trainset/dicom/abn_180517_1/33295029_0003.dcm
/home/huray/data/NCC_trainset/dicom/abn_180517_1/33295029_0004.dcm
/home/huray/data/NCC_trainset/dicom/abn_180517_1/33295039_0001.dcm
/home/huray/data/NCC_trainset/dicom/abn_180517_1/33295039_0002.dcm
/home/huray/data/NCC_trainset/dicom/abn_180517_1/33295039_0003

/home/huray/data/NCC_trainset/dicom/abn_180517_2/33267015_0003.dcm
/home/huray/data/NCC_trainset/dicom/abn_180517_2/33267015_0004.dcm
/home/huray/data/NCC_trainset/dicom/abn_180517_2/33267039_0001.dcm
/home/huray/data/NCC_trainset/dicom/abn_180517_2/33267039_0002.dcm
/home/huray/data/NCC_trainset/dicom/abn_180517_2/33267039_0003.dcm
/home/huray/data/NCC_trainset/dicom/abn_180517_2/33267039_0004.dcm
/home/huray/data/NCC_trainset/dicom/abn_180517_2/33267290_0001.dcm
/home/huray/data/NCC_trainset/dicom/abn_180517_2/33267290_0002.dcm
/home/huray/data/NCC_trainset/dicom/abn_180517_2/33267290_0003.dcm
/home/huray/data/NCC_trainset/dicom/abn_180517_2/33267290_0004.dcm
/home/huray/data/NCC_trainset/dicom/abn_180517_2/33268222_0001.dcm
/home/huray/data/NCC_trainset/dicom/abn_180517_2/33268222_0002.dcm
/home/huray/data/NCC_trainset/dicom/abn_180517_2/33268222_0003.dcm
/home/huray/data/NCC_trainset/dicom/abn_180517_2/33268222_0004.dcm
/home/huray/data/NCC_trainset/dicom/abn_180517_2/33269521_0001

/home/huray/data/NCC_trainset/dicom/abn_180517_2/33281797_0002.dcm
/home/huray/data/NCC_trainset/dicom/abn_180517_2/33281797_0003.dcm
/home/huray/data/NCC_trainset/dicom/abn_180517_2/33281797_0004.dcm
/home/huray/data/NCC_trainset/dicom/abn_180517_2/33281855_0001.dcm
/home/huray/data/NCC_trainset/dicom/abn_180517_2/33281855_0002.dcm
/home/huray/data/NCC_trainset/dicom/abn_180517_2/33281855_0003.dcm
/home/huray/data/NCC_trainset/dicom/abn_180517_2/33281855_0004.dcm
/home/huray/data/NCC_trainset/dicom/abn_180517_2/33281943_0001.dcm
/home/huray/data/NCC_trainset/dicom/abn_180517_2/33281943_0002.dcm
/home/huray/data/NCC_trainset/dicom/abn_180517_2/33281943_0003.dcm
/home/huray/data/NCC_trainset/dicom/abn_180517_2/33281943_0004.dcm
/home/huray/data/NCC_trainset/dicom/abn_180517_2/33282061_0001.dcm
/home/huray/data/NCC_trainset/dicom/abn_180517_2/33282061_0002.dcm
/home/huray/data/NCC_trainset/dicom/abn_180517_2/33282061_0003.dcm
/home/huray/data/NCC_trainset/dicom/abn_180517_2/33282061_0004

/home/huray/data/NCC_trainset/dicom/abn_180517_2/33285059_0004.dcm
/home/huray/data/NCC_trainset/dicom/abn_180517_2/33285134_0001.dcm
/home/huray/data/NCC_trainset/dicom/abn_180517_2/33285134_0002.dcm
/home/huray/data/NCC_trainset/dicom/abn_180517_2/33285134_0003.dcm
/home/huray/data/NCC_trainset/dicom/abn_180517_2/33285134_0004.dcm
/home/huray/data/NCC_trainset/dicom/abn_180517_2/33285638_0001.dcm
/home/huray/data/NCC_trainset/dicom/abn_180517_2/33285638_0002.dcm
/home/huray/data/NCC_trainset/dicom/abn_180517_2/33285638_0003.dcm
/home/huray/data/NCC_trainset/dicom/abn_180517_2/33285638_0004.dcm
/home/huray/data/NCC_trainset/dicom/abn_180517_2/33285669_0001.dcm
/home/huray/data/NCC_trainset/dicom/abn_180517_2/33285669_0002.dcm
/home/huray/data/NCC_trainset/dicom/abn_180517_2/33285669_0003.dcm
/home/huray/data/NCC_trainset/dicom/abn_180517_2/33285669_0004.dcm
/home/huray/data/NCC_trainset/dicom/abn_180517_2/33285762_0001.dcm
/home/huray/data/NCC_trainset/dicom/abn_180517_2/33285762_0002

In [7]:
print("ERRORED DATA COUNT: ", errored_data_counter)
print("FILES WITH WRONG SIZE COUNT: ", len(files_with_wrong_size))
print("FILES WITH WRONG SIZE: ", files_with_wrong_size)

ERRORED DATA COUNT:  0
FILES WITH WRONG SIZE COUNT:  0
FILES WITH WRONG SIZE:  []


In [8]:
print(len(img_path), len(x1), len(y1), len(x2), len(y2), len(class_name))

905 905 905 905 905 905


In [9]:
df = pd.DataFrame({'img_path':img_path, 'x1':x1, 'y1':y1, 'x2':x2, 'y2':y2, 'class_name':class_name})
df = df[['img_path', 'x1', 'y1', 'x2', 'y2', 'class_name']]
print(len(df))

905


In [10]:
df.head()

,img_path,x1,y1,x2,y2,class_name
0,/home/huray/data/NCC_trainset/img_retinanet/ab...,,,,,
1,/home/huray/data/NCC_trainset/img_retinanet/ab...,307,1269,401,1365,M
2,/home/huray/data/NCC_trainset/img_retinanet/ab...,201,1074,260,1126,M
3,/home/huray/data/NCC_trainset/img_retinanet/ab...,629,949,702,1032,M
4,/home/huray/data/NCC_trainset/img_retinanet/ab...,652,846,737,923,M


In [11]:
print(df['img_path'][0])
print(df['img_path'][1])
print(df['img_path'][2])
print(df['img_path'][3])

/home/huray/data/NCC_trainset/img_retinanet/abn_180517_1/20000686_0001_0.jpg
/home/huray/data/NCC_trainset/img_retinanet/abn_180517_1/20000686_0002_0.jpg
/home/huray/data/NCC_trainset/img_retinanet/abn_180517_1/20000686_0002_1.jpg
/home/huray/data/NCC_trainset/img_retinanet/abn_180517_1/20000686_0002_2.jpg


In [12]:
df.to_csv('/home/huray/data/NCC_trainset/img_retinanet/data_ncc_abn_180517.csv', header=False, index=False)
# df.to_csv('/home/huray/data/NCC_trainset/img_retinanet/data_normal.csv', header=False, index=False)